### Universidad del Valle de Guatemala
### Modelado y Simulacion
#### MICRO PROYECTO 4
##### Juan Marroquin 19845
##### Carlos Raxtum 19721

import libraries

In [1]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

#### Seed para generador de númerosa aleatorios uniformes

In [2]:
random.seed(8000)

#### Generador de Numeros Aleatorios

In [3]:
def Exponential(Lambda):
    return -float((1./Lambda)*math.log(random.random()))

In [9]:
def Poisson(s,Lambda):
    return s - (1/Lambda)*math.log(random.random())

#### MountainMegaComputing simulator

In [10]:
def MountainMegaComputing(num_sol=2400.,tiempo_benchMark=3600):
    inf = float('inf')
    solicitudesXsegundo = num_sol/60
    t,num_A,num_D,t_arrival,t_departures,n,time_process = 0,0,0,0,0,0,0
    T = tiempo_benchMark # final time
    arrivals,departures = [],[]
    while t<=T:
        if(t_arrival<=t_departures and t_arrival<=T):
            t = t_arrival
            num_A = num_A + 1
            n = n + 1
            t_arrival = Poisson(t,solicitudesXsegundo)
            if n==1:
                Y = Exponential(solicitudesXsegundo)
                t_departures = t + Y
            arrivals.append(t)
        elif(t_departures<t_arrival and t_departures<=T):
            t = t_departures
            n = n - 1
            num_D = num_D +1
            if n == 0:
                t_departures = inf
            else:
                Y = Exponential(solicitudesXsegundo)
                t_departures = t + Y
                t = t_departures
            departures.append(t)
        
        elif(min(t_arrival,t_departures) > T and n > 0):
            t =t_departures
            n = n - 1
            num_D = num_D + 1
            if n>0:
                Y = Exponential(solicitudesXsegundo)
                t_departures = t + Y
                t = t_departures
            departures.append(t)
        elif(min(t_arrival,t_departures) >T and n == 0):
            time_process = max(t - T,0)
            
            break
    
    total = 0
    for k in range(len(departures)):
        total += departures[k]-arrivals[k]
    prom = total/len(arrivals)
    print("ESTE ES EL RESULTADO PARA EL SERVIDOR Mountain Mega Computing")
    print("El numero de llegadas al servidor fue de: ", num_A)
    print("El tiempo que estuvo ocupado el servidor fue: ", round(t_departures,2))
    print("El tiempo que estuvo libre el servidor fue de: ", round(t_departures - t_arrival,2))
    print("El tiempo en total estuvieron las solicitudes en cola: ",round(total,2))
    print("El tiempo promedio que estuvo cada solitud en cola: ",round(prom ,2))
    print("El numero de solicitudes que estuvieron en cola cada segundo: ", round(len(arrivals)/60,2))
    print("El numero de solicitudes en cola al momento de terminar la simulacion: ",round(len(arrivals)-len(departures),2))

#### PizzaComputing simulator

In [31]:
def PizzaComputing(num_server,num_sol=2400.,tiempo_benchMark=3600,show=True):
    inf = float('inf')                             
    T = tiempo_benchMark                                                         
    num_A,t = 0,0
    arrivals,departures,server = [],[],[]                                           
    solicitudesXsegundo = num_sol/60
    t_arrival = Exponential(solicitudesXsegundo)             
    t_departures = [100000]*num_server                   
    process_servers = [0]*num_server                        
    solicitude_Server = [0]*(1+num_server)                    
    
    while t < T or solicitude_Server[0] > 0:           
        m = min(t_departures)
        min_time = t_departures.index(m)            
        
        if t_arrival <= t_departures[min_time] and t_arrival<= T:   
            t = t_arrival                      
            num_A = num_A +  1                
            t_arrival = t_arrival + Exponential(solicitudesXsegundo)
            arrivals.append(t)                   
            
            if solicitude_Server[0] == 0:              
                server.append(t)            
                solicitude_Server[0] += 1              
                solicitude_Server[1] = num_A              
                t_departures[0] = t + Exponential(10) 
                
            elif solicitude_Server[0] < num_server:            
                busy = solicitude_Server.index(0) - 1 
                server.append(t)             
                solicitude_Server[0] += 1              
                solicitude_Server[busy+1] = num_A      
                t_departures[busy] = t + Exponential(10) 
            
            else:                      
                solicitude_Server[0] += 1              
            
        else:                           
            t = t_departures[min_time]              
            process_servers[min_time] += 1             
            departures.append(t)                 
            
            if solicitude_Server[0] <= num_server:            
                solicitude_Server[0] -= 1              
                t_departures[min_time] = 100000     
                solicitude_Server[min_time + 1] = 0      
            
            else:                       
                solicitude_Server[0] *= -1
                next_evente = max(solicitude_Server)+1   
                solicitude_Server[0] *= -1
                server.append(t)                             
                solicitude_Server[0] -= 1              
                t_departures[min_time] = t + Exponential(10)
                solicitude_Server[min_time + 1] = next_evente 
    total=0
    for k in range(len(arrivals)):
        resta=server[k]-arrivals[k]
        total=total+resta
    prom = total/len(arrivals)
    if show is True:
        print("Solicitudes atendididas por cada servidor:")
        print("-----------------------------------------")
        for i in range (len(process_servers)):
            print("\t"+"Servidor "+str(i+1)+":", process_servers[i], "solicitudes.")
        print("-----------------------------------------")
        print("Tiempo total: ", sum(t_departures), "segundos.")
        print("Solicitudes atendidas: ", num_A)
        print("Ultima llegada:", round(arrivals[-1],2), "s.")
        print("Ultima salida:", round(departures[-1],2), "s.")
        print("-----------------------------------------\n")
    print("ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing")
    print("El numero de llegadas al servidor fue de: ", num_A)
    print("El tiempo que estuvo ocupado el servidor fue: ", round(sum(t_departures),2))
    print("El tiempo que estuvo libre el servidor fue de: ", round(sum(t_departures) - t_arrival,2))
    print("El tiempo en total estuvieron las solicitudes en cola: ",round(total,2))
    print("El tiempo promedio que estuvo cada solitud en cola: ",round(prom ,2))
    print("El numero de solicitudes que estuvieron en cola cada segundo: ", round(len(arrivals)/60,2))
    print("El numero de solicitudes en cola al momento de terminar la simulacion: ",round(len(departures)-len(arrivals),2))
    print("El numero de solicitudes en ejecutadas por el ultimo servidor: ",process_servers[-1])
    #print(len(arrivals),len(departures),len(server),num_A)
    

#### Task 1
#### Modele, simule y analice el comportamiento de ambos sistemas durante una hora de ejecución

#### Mountain Mega Computing

In [20]:
MountainMegaComputing()

ESTE ES EL RESULTADO PARA EL SERVIDOR Mountain Mega Computing
El numero de llegadas al servidor fue de:  144234
El tiempo que estuvo ocupado el servidor fue:  3600.0
El tiempo que estuvo libre el servidor fue de:  -0.0
El tiempo en total estuvieron las solicitudes en cola:  1942715.63
El tiempo promedio que estuvo cada solitud en cola:  13.47
El numero de solicitudes que estuvieron en cola cada segundo:  2403.9
El numero de solicitudes en cola al momento de terminar la simulacion:  523


#### Pizza Computing

In [32]:
PizzaComputing(1,show=True)

Solicitudes atendididas por cada servidor:
-----------------------------------------
	Servidor 1: 143186 solicitudes.
-----------------------------------------
Tiempo total:  100000 segundos.
Solicitudes atendidas:  143186
Ultima llegada: 3599.97 s.
Ultima salida: 14392.07 s.
-----------------------------------------

ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing
El numero de llegadas al servidor fue de:  143186
El tiempo que estuvo ocupado el servidor fue:  100000
El tiempo que estuvo libre el servidor fue de:  96399.99
El tiempo en total estuvieron las solicitudes en cola:  771686079.48
El tiempo promedio que estuvo cada solitud en cola:  5389.4
El numero de solicitudes que estuvieron en cola cada segundo:  2386.43
El numero de solicitudes en cola al momento de terminar la simulacion:  0
El numero de solicitudes en ejecutadas por el ultimo servidor:  143186


#### Task 2
#### Determine empíricamente cuántos servidores se necesitaría “alquilar” en Pizzita computing para asegurar que siempre habrá al menos un servidor disponible para atender una solicitud dada

In [33]:
for i in range(1,21):
    print("-----------------------------------------")
    print('PARA ',i,' SERVIDORES')
    PizzaComputing(i,show=False)
    print("-----------------------------------------\n")

-----------------------------------------
PARA  1  SERVIDORES
ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing
El numero de llegadas al servidor fue de:  144382
El tiempo que estuvo ocupado el servidor fue:  100000
El tiempo que estuvo libre el servidor fue de:  96399.98
El tiempo en total estuvieron las solicitudes en cola:  782243947.99
El tiempo promedio que estuvo cada solitud en cola:  5417.88
El numero de solicitudes que estuvieron en cola cada segundo:  2406.37
El numero de solicitudes en cola al momento de terminar la simulacion:  0
El numero de solicitudes en ejecutadas por el ultimo servidor:  144382
-----------------------------------------

-----------------------------------------
PARA  2  SERVIDORES
ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing
El numero de llegadas al servidor fue de:  143812
El tiempo que estuvo ocupado el servidor fue:  200000
El tiempo que estuvo libre el servidor fue de:  196399.96
El tiempo en total estuvieron las solicitudes en cola:  

#### Task 3
#### Se espera que apartir del tercer año del lanzamiento de su aplicación , la cantidad de usuarios sufra un alza, y por tanto deberán atender como máximo 6000 solicitudes por minuto

#### Mountain Mega Computing

In [23]:
MountainMegaComputing(num_sol=6000)

ESTE ES EL RESULTADO PARA EL SERVIDOR Mountain Mega Computing
El numero de llegadas al servidor fue de:  359246
El tiempo que estuvo ocupado el servidor fue:  3600.02
El tiempo que estuvo libre el servidor fue de:  0.06
El tiempo en total estuvieron las solicitudes en cola:  514664.66
El tiempo promedio que estuvo cada solitud en cola:  1.43
El numero de solicitudes que estuvieron en cola cada segundo:  5987.43
El numero de solicitudes en cola al momento de terminar la simulacion:  82


#### Pizza Computing

In [38]:
for i in range(20,31):
    print("-----------------------------------------")
    print('PARA ',i,' SERVIDORES')
    PizzaComputing(i,show=True,num_sol=6000)
    print("-----------------------------------------\n")
#PizzaComputing(29,show=True,num_sol=6000)

-----------------------------------------
PARA  20  SERVIDORES
Solicitudes atendididas por cada servidor:
-----------------------------------------
	Servidor 1: 32661 solicitudes.
	Servidor 2: 32350 solicitudes.
	Servidor 3: 31386 solicitudes.
	Servidor 4: 30862 solicitudes.
	Servidor 5: 29767 solicitudes.
	Servidor 6: 28575 solicitudes.
	Servidor 7: 27118 solicitudes.
	Servidor 8: 25608 solicitudes.
	Servidor 9: 23555 solicitudes.
	Servidor 10: 21213 solicitudes.
	Servidor 11: 18545 solicitudes.
	Servidor 12: 15724 solicitudes.
	Servidor 13: 12763 solicitudes.
	Servidor 14: 10041 solicitudes.
	Servidor 15: 7484 solicitudes.
	Servidor 16: 5148 solicitudes.
	Servidor 17: 3442 solicitudes.
	Servidor 18: 2207 solicitudes.
	Servidor 19: 1309 solicitudes.
	Servidor 20: 713 solicitudes.
-----------------------------------------
Tiempo total:  2000000 segundos.
Solicitudes atendidas:  360471
Ultima llegada: 3599.99 s.
Ultima salida: 3600.27 s.
-----------------------------------------

ESTE E

#### Se recomienda Contratar 30 servidores para Pizza Computing para 6000 solicitudes

#### Task 4
#### Emita una recomendación para la junta directiva

* Si se tiene un presupuesto mayor, les recomendaria contratar Pizza Computing Servers, con 30 servidores, ya que es capaz de atender una gran cantidad de peticiones sin ninguno problema y evitaria que el cliente se quede esperendo a ser atendido.
* Si el presupuesto no es lo sufciente, la opcion de Mountain Mega Computing Servers, ya que es una opcion mas economica, pero esta penalizaria al usuario ya que tendria que esperar un tiempo para ser atendido

* Si para el inicio del proyecto no se cuenta con un gran presupeusto pero se espera que al crecer el negocio el presupuesto aumente se puede iniciar con Mountain Mega Computing y luego mudarse luego a Pizza Computing.
* En conclusion Pizza Computing es la mejor opcion desde un incio pero no es economica ya que cada alquiler de un servidor es muy costoso y todo depende de presupuesto del proyecto